# Two-Stage Object detection w/ Freeze Learning & Fine-tuning
For this approach, it will be used a pretrained DensetNet201 as a backbone for a F-RCNN model for object detection, to which it will be applied freeze learning and fine-tuning to our task.

In [32]:
import os
import torch

DATA_DIR = "../data/"
IMG_DIR = DATA_DIR + "/images/"
ANNOTATION_DIR = DATA_DIR + "/annotations/"
SPLITS_DIR = DATA_DIR + "/dl-split/"
OUT_DIR = "./out/yolo_obj_detect/"

os.makedirs(OUT_DIR, exist_ok=True)

SEED = 42

# Model Definition

In [33]:
!git clone https://github.com/ultralytics/yolov5 models/yolov5

fatal: destination path 'models/yolov5' already exists and is not an empty directory.


# Train-Validation Split

In [34]:
import numpy as np

np.random.seed(SEED)

training_filenames = []
with open('../data/dl-split/train.txt', 'r') as train_f:
    training_filenames = train_f.readlines()
    training_filenames = [x.strip() for x in training_filenames]
        

train_indices = list(range(len(training_filenames)))
np.random.shuffle(train_indices)
train_val_split = int(np.floor(0.2 * len(train_indices)))

train_idx, val_idx = train_indices[train_val_split:], train_indices[:train_val_split]

train_data = [training_filenames[idx] for idx in train_idx]
val_data = [training_filenames[idx] for idx in val_idx]

test_data = []
with open('../data/dl-split/test.txt', 'r') as test_f:
    test_data = test_f.readlines()
    test_data = [x.strip() for x in test_data]
    


# Prepare Folders

In [35]:
from utils.utils import parse_annotation
import shutil

TRAIN_DIR='../data/images/train'
TEST_DIR='../data/images/test'
VAL_DIR='../data/images/val'

TRAIN_LABELS='../data/labels/train'
TEST_LABELS='../data/labels/test'
VAL_LABELS='../data/labels/val'

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)

os.makedirs(TRAIN_LABELS, exist_ok=True)
os.makedirs(TEST_LABELS, exist_ok=True)
os.makedirs(VAL_LABELS, exist_ok=True)

label_encode_map = {
    "trafficlight": 0,
    "speedlimit": 1,
    "crosswalk": 2,
    "stop": 3,
}

def transform_labels(out_path: str, annot_dict):
    annot_file = open(out_path, 'w')
    
    img_width = annot_dict['width']
    img_height = annot_dict['height']
    for label, box in zip(annot_dict['labels'], annot_dict['boxes']):
        width = (box[2]-box[0])/img_width
        height = (box[3]-box[1])/img_height
        
        x_center = (box[0]+box[2])/2/img_width
        y_center = (box[3]+box[1])/2/img_height
        
        annot_file.write(f'{label} {x_center} {y_center} {width} {height}\n')
    annot_file.close()

# train files
for name in train_data:
    annot_dict = parse_annotation(f'../data/annotations/{name}.xml', label_encode_map)
    transform_labels(f'{TRAIN_LABELS}/{name}.txt', annot_dict)
    shutil.copy(src=f'../data/images/{name}.png', dst=TRAIN_DIR)
    
# validation files
for name in val_data:
    annot_dict = parse_annotation(f'../data/annotations/{name}.xml', label_encode_map)
    transform_labels(f'{VAL_LABELS}/{name}.txt', annot_dict)
    shutil.copy(src=f'../data/images/{name}.png', dst=VAL_DIR)
    
# test files 
for name in test_data:
    annot_dict = parse_annotation(f'../data/annotations/{name}.xml', label_encode_map)
    transform_labels(f'{TEST_LABELS}/{name}.txt', annot_dict)
    shutil.copy(src=f'../data/images/{name}.png', dst=TEST_DIR)

# Train Model

In [39]:
!python models/yolov5/train.py -h

usage: train.py [-h] [--weights WEIGHTS] [--cfg CFG] [--data DATA] [--hyp HYP]
                [--epochs EPOCHS] [--batch-size BATCH_SIZE] [--imgsz IMGSZ]
                [--rect] [--resume [RESUME]] [--nosave] [--noval]
                [--noautoanchor] [--noplots] [--evolve [EVOLVE]]
                [--bucket BUCKET] [--cache [CACHE]] [--image-weights]
                [--device DEVICE] [--multi-scale] [--single-cls]
                [--optimizer {SGD,Adam,AdamW}] [--sync-bn] [--workers WORKERS]
                [--project PROJECT] [--name NAME] [--exist-ok] [--quad]
                [--cos-lr] [--label-smoothing LABEL_SMOOTHING]
                [--patience PATIENCE] [--freeze FREEZE [FREEZE ...]]
                [--save-period SAVE_PERIOD] [--local_rank LOCAL_RANK]
                [--entity ENTITY] [--upload_dataset [UPLOAD_DATASET]]
                [--bbox_interval BBOX_INTERVAL]
                [--artifact_alias ARTIFACT_ALIAS]

options:
  -h, --help            show this help message a

In [37]:
!python models/yolov5/train.py --batch 4 --epochs 10 --data models/yolo_cfg.yaml --workers 2

train: weights=models\yolov5\yolov5s.pt, cfg=, data=models/yolo_cfg.yaml, hyp=models\yolov5\data\hyps\hyp.scratch-low.yaml, epochs=3, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=models\yolov5\runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 
fatal: cannot change to 'C:\Users\Ivo': No such file or directory
YOLOv5  2022-6-8 Python-3.10.1 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce GTX 950, 2048MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5